# Retrieval Application

## Vector Database (Vector DB)
Resources 
- [How-to guides](https://python.langchain.com/v0.2/docs/how_to/#vector-stores)
  - [Vectorstores](https://python.langchain.com/v0.2/docs/integrations/vectorstores/): A vector store that stores embedded data and performs similarity search.
    1. [Elasticsearch](https://python.langchain.com/v0.2/docs/integrations/vectorstores/elasticsearch/)
    2. [Milvus](https://python.langchain.com/v0.2/docs/integrations/vectorstores/milvus/)
    3. [Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)

# Environment Setup

Sources  
- [langchain-chroma](https://pypi.org/project/langchain-chroma/)

In [20]:
from importlib.metadata import version
# !pip install langchain
# Select langchain to 0.1.20
try:
    print('langchain package version',version('langchain'))
    assert version('langchain') == '0.1.20'
except:
    !pip install langchain==0.1.20
    
# !pip install --upgrade langchain
# print('langchain package version',version('langchain'))

langchain package version 0.1.20


In [21]:
#!pip install -qU langchain-huggingface
# Select langchain-huggingface to 0.0.3
try:
    print('langchain-huggingface package version',version('langchain-huggingface'))
    assert version('langchain-huggingface') =='0.0.3'#'0.2.11'
except:
    !pip install langchain-huggingface==0.0.3
    #0.2.11 (if any)
    
# !pip install -qU langchain-huggingface
# print('langchain-huggingface package version',version('langchain-huggingface'))

langchain-huggingface package version 0.0.3


In [22]:
# Select langchain-chroma to 0.1.3
try:
    print('langchain_chroma package version',version('langchain_chroma'))
    assert version('langchain_chroma') == '0.1.3'
except:
    !pip install langchain_chroma==0.1.3
    
# !pip install -qU langchain_chroma==0.1.3
# print('langchain_chroma package version',version('langchain_chroma'))

langchain_chroma package version 0.1.3


In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import os
os.getcwd()

'/work/files/workspace'

In [24]:
import langchain_chroma 
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Import Data
- Data source: [LinkedIn Job Postings (2023 - 2024)](https://www.kaggle.com/datasets/arshkon/linkedin-job-postings)

**Features of Use**
- id:
  1. job_id  

- Main data  

  **Texts**  
  1. _description_
  1. _skills_desc_  

  **Vector**  
  1. title
  2. _description_
  3. _skills_desc_  
  
- Meta data:  
  1. title
  1. location
  1. min_salary
  1. pay_period
  1. job_posting_url
  
  


In [25]:
df = pd.read_csv('./Data/archive/postings.csv')

print(f"The variables (features) of the data:\n{df.columns}")
df.head(5)

The variables (features) of the data:
Index(['job_id', 'company_name', 'title', 'description', 'max_salary',
       'pay_period', 'location', 'company_id', 'views', 'med_salary',
       'min_salary', 'formatted_work_type', 'applies', 'original_listed_time',
       'remote_allowed', 'job_posting_url', 'application_url',
       'application_type', 'expiry', 'closed_time',
       'formatted_experience_level', 'skills_desc', 'listed_time',
       'posting_domain', 'sponsored', 'work_type', 'currency',
       'compensation_type', 'normalized_salary', 'zip_code', 'fips'],
      dtype='object')


,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,min_salary,formatted_work_type,applies,original_listed_time,remote_allowed,job_posting_url,application_url,application_type,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,17.0,Full-time,2.0,1.713398e+12,NaN,https://www.linkedin.com/jobs/view/921716/?trk...,NaN,ComplexOnsiteApply,1.715990e+12,NaN,NaN,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,30.0,Full-time,NaN,1.712858e+12,NaN,https://www.linkedin.com/jobs/view/1829192/?tr...,NaN,ComplexOnsiteApply,1.715450e+12,NaN,NaN,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,45000.0,Full-time,NaN,1.713278e+12,NaN,https://www.linkedin.com/jobs/view/10998357/?t...,NaN,ComplexOnsiteApply,1.715870e+12,NaN,NaN,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,140000.0,Full-time,NaN,1.712896e+12,NaN,https://www.linkedin.com/jobs/view/23221523/?t...,NaN,ComplexOnsiteApply,1.715488e+12,NaN,NaN,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,60000.0,Full-time,NaN,1.713452e+12,NaN,https://www.linkedin.com/jobs/view/35982263/?t...,NaN,ComplexOnsiteApply,1.716044e+12,NaN,NaN,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0


In [26]:
# df['job_post'] = df['description'].astype('str') + df['skills_desc'].astype('str')
# df[['description', 'skills_desc', 'job_post']].head()

In [27]:
df['job_post'] = np.where(df['skills_desc'].isna(), df['description'].astype('str'), df['skills_desc'].astype('str'))
df[['description', 'skills_desc', 'job_post']].head()

,description,skills_desc,job_post
0,Job descriptionA leading real estate firm in N...,Requirements: \n\nWe are seeking a College or ...,Requirements: \n\nWe are seeking a College or ...
1,"At Aspen Therapy and Wellness , we are committ...",NaN,"At Aspen Therapy and Wellness , we are committ..."
2,The National Exemplar is accepting application...,We are currently accepting resumes for FOH - A...,We are currently accepting resumes for FOH - A...
3,Senior Associate Attorney - Elder Law / Trusts...,This position requires a baseline understandin...,This position requires a baseline understandin...
4,Looking for HVAC service tech with experience ...,NaN,Looking for HVAC service tech with experience ...


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 32 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      123849 non-null  int64  
 1   company_name                122130 non-null  object 
 2   title                       123849 non-null  object 
 3   description                 123842 non-null  object 
 4   max_salary                  29793 non-null   float64
 5   pay_period                  36073 non-null   object 
 6   location                    123849 non-null  object 
 7   company_id                  122132 non-null  float64
 8   views                       122160 non-null  float64
 9   med_salary                  6280 non-null    float64
 10  min_salary                  29793 non-null   float64
 11  formatted_work_type         123849 non-null  object 
 12  applies                     23320 non-null   float64
 13  original_liste

## Select Data
**Purpose**:
- Insert data samples into Vector Database (VectorDB), working as knowledge base

**Note**:
- For demonstration, I pick 50% of data related to a specific job tiltes, and 50% other random job titles

In [29]:
# define yourself

## job title you're interested
keywords = ['data science', 'data scientist', 'data analyst'] 
# keywords = ['data analytics', 'data scientist', 'hr analytics', 'hr data analyst']

## Number of relevant / irrelevant samples
N = 400

In [30]:
condition = df['title'].str.contains('|'.join(keywords), case=False, na=False)
df_ds = df[condition]
# 'case=False' makes the search case-insensitive
# 'na=False' ensures that NaN values are not considered in the search

N_ds = min(len(df_ds), N)
df_ds = df_ds.sample(n=N_ds)
print(f"There're {N_ds} samples searched according to keywords.")
df_ds.head(10)

There're 400 samples searched according to keywords.


,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,min_salary,formatted_work_type,applies,original_listed_time,remote_allowed,job_posting_url,application_url,application_type,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips,job_post
3660,3884844890,JPMorgan Chase & Co.,Marketing Analytics - Data Scientist Senior As...,Job Description\n\nJoin a talented and diverse...,NaN,NaN,"Columbus, OH",1068.0,5.0,NaN,NaN,Full-time,NaN,1.712376e+12,NaN,https://www.linkedin.com/jobs/view/3884844890/...,https://JPMorganChase.contacthr.com/136085025,OffsiteApply,1.714969e+12,NaN,NaN,NaN,1.712377e+12,JPMorganChase.contacthr.com,0,FULL_TIME,NaN,NaN,NaN,43085.0,39049.0,Job Description\n\nJoin a talented and diverse...
75605,3903454514,EPITEC,Data Analyst,JOB SUMMARY FOR PRODUCT ANALYST:\nNO C2CThe GT...,73.0,HOURLY,"Chicago, IL",25461.0,411.0,NaN,68.0,Contract,185.0,1.713455e+12,NaN,https://www.linkedin.com/jobs/view/3903454514/...,NaN,ComplexOnsiteApply,1.716047e+12,NaN,Mid-Senior level,NaN,1.713455e+12,NaN,0,CONTRACT,USD,BASE_SALARY,146640.0,60601.0,17031.0,JOB SUMMARY FOR PRODUCT ANALYST:\nNO C2CThe GT...
86454,3904383795,Northwestern Mutual,Sr Data Analyst,"At Northwestern Mutual, we are strong, innovat...",139620.0,YEARLY,"Franklin, WI",2445.0,42.0,NaN,75180.0,Full-time,8.0,1.713392e+12,NaN,https://www.linkedin.com/jobs/view/3904383795/...,https://careers.northwesternmutual.com/corpora...,OffsiteApply,1.715985e+12,NaN,Mid-Senior level,NaN,1.713393e+12,careers.northwesternmutual.com,0,FULL_TIME,USD,BASE_SALARY,107400.0,53132.0,55079.0,"At Northwestern Mutual, we are strong, innovat..."
20815,3889288309,Recursion,Staff Data Scientist,Your work will change lives. Including your ow...,NaN,NaN,"Salt Lake City, UT",4990022.0,5.0,NaN,NaN,Full-time,NaN,1.712651e+12,NaN,https://www.linkedin.com/jobs/view/3889288309/...,https://boards.greenhouse.io/recursionpharmace...,OffsiteApply,1.715243e+12,NaN,Entry level,NaN,1.712651e+12,boards.greenhouse.io,0,FULL_TIME,NaN,NaN,NaN,84101.0,49035.0,Your work will change lives. Including your ow...
117069,3905879216,ClearanceJobs,Data Science Manager - GPS with Security Clear...,"The SFL Scientific, a Deloitte Business practi...",154000.0,YEARLY,"Colorado Springs, CO",1007957.0,3.0,NaN,92000.0,Full-time,NaN,1.713485e+12,NaN,https://www.linkedin.com/jobs/view/3905879216/...,https://click.appcast.io/track/icdi8hz?cs=i8d&...,OffsiteApply,1.716133e+12,NaN,Mid-Senior level,NaN,1.713541e+12,click.appcast.io,0,FULL_TIME,USD,BASE_SALARY,123000.0,80902.0,NaN,"The SFL Scientific, a Deloitte Business practi..."
72933,3902914840,Tetricus Labs,Data Scientist,Our Mission\nTetricus is at the forefront of m...,160000.0,YEARLY,United States,85401549.0,436.0,NaN,120000.0,Full-time,204.0,1.713277e+12,1.0,https://www.linkedin.com/jobs/view/3902914840/...,NaN,ComplexOnsiteApply,1.728829e+12,NaN,NaN,NaN,1.713277e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,140000.0,NaN,NaN,Our Mission\nTetricus is at the forefront of m...
6733,3885811236,Capital One,"Senior Manager, Data Science - Business Card &...","NYC 299 Park Avenue (22957), United States of ...",NaN,NaN,"Cambridge, MA",1419.0,7.0,NaN,NaN,Full-time,NaN,1.712366e+12,NaN,https://www.linkedin.com/jobs/view/3885811236/...,https://dsp.prng.co/7k3yx5b,OffsiteApply,1.714959e+12,NaN,Mid-Senior level,NaN,1.712367e+12,dsp.prng.co,0,FULL_TIME,NaN,NaN,NaN,2138.0,25017.0,"NYC 299 Park Avenue (22957), United States of ..."
55936,3901937364,DataAnnotation,Data Analyst,DataAnnotation is committed to creating qualit...,NaN,NaN,Greater Houston,96139831.0,4.0,NaN,NaN,Contract,NaN,1.713466e+12,1.0,https://www.linkedin.com/jobs/view/3901937364/...,https://app.dataannotation.tech/worker_signup?...,OffsiteApply,1.716058e+12,NaN,NaN,NaN,1.713466e+12,NaN,0,CONTRACT,NaN,NaN,NaN,NaN,NaN,DataAnnotation is committed to creating qualit...
32838,3894954189,Educ

In [31]:
df_others = df[~condition].sample(n=N)
print(f"There're {len(df_others)} samples searched not contained in keywords.")
df_others.head(10)

There're 400 samples searched not contained in keywords.


,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,min_salary,formatted_work_type,applies,original_listed_time,remote_allowed,job_posting_url,application_url,application_type,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips,job_post
43067,3900068690,Dexian,Tax Accountant,Accounting firm in business over 30 years is n...,NaN,NaN,"Hollywood, FL",91447303.0,10.0,NaN,NaN,Full-time,3.0,1.713208e+12,NaN,https://www.linkedin.com/jobs/view/3900068690/...,NaN,SimpleOnsiteApply,1.715800e+12,NaN,Associate,NaN,1.713208e+12,NaN,0,FULL_TIME,NaN,NaN,NaN,33019.0,12011.0,Accounting firm in business over 30 years is n...
88960,3904404876,Theoria Medical,Medical Director,"Position Type: Part-time, exempt\n\nCompensati...",NaN,NaN,"Petersburg, IN",51664810.0,6.0,NaN,NaN,Part-time,NaN,1.713312e+12,NaN,https://www.linkedin.com/jobs/view/3904404876/...,https://boards.greenhouse.io/theoriamedical/jo...,ComplexOnsiteApply,1.715992e+12,NaN,Director,NaN,1.713400e+12,boards.greenhouse.io,0,PART_TIME,NaN,NaN,NaN,47567.0,18125.0,"Position Type: Part-time, exempt\n\nCompensati..."
50963,3901377609,Resource Logistics Inc.,Medical Assistant,Job Title: Medical AssistantDuration: 13 weeks...,NaN,NaN,"Beverly Hills, CA",107021.0,9.0,NaN,NaN,Contract,NaN,1.713459e+12,NaN,https://www.linkedin.com/jobs/view/3901377609/...,NaN,ComplexOnsiteApply,1.716051e+12,NaN,NaN,NaN,1.713459e+12,NaN,0,CONTRACT,NaN,NaN,NaN,90210.0,6037.0,Job Title: Medical AssistantDuration: 13 weeks...
3701,3884848517,UL Solutions,Senior Writer - Corporate Communications,Job Description\n\nThis role is hybrid based (...,NaN,NaN,"Chicago, IL",4721.0,8.0,NaN,NaN,Full-time,NaN,1.712275e+12,NaN,https://www.linkedin.com/jobs/view/3884848517/...,https://fa-eups-saasfaprod1.fa.ocs.oraclecloud...,OffsiteApply,1.714969e+12,NaN,Mid-Senior level,NaN,1.712377e+12,fa-eups-saasfaprod1.fa.ocs.oraclecloud.com,0,FULL_TIME,NaN,NaN,NaN,60601.0,17031.0,Job Description\n\nThis role is hybrid based (...
28224,3891832806,The Job Network,Registered Nurse (RN) or Licensed Practical Nu...,"Since 2012, Giving Home Health Care has helped...",NaN,NaN,"Tucson, AZ",167757.0,2.0,NaN,NaN,Full-time,NaN,1.712904e+12,NaN,https://www.linkedin.com/jobs/view/3891832806/...,https://givinghomehealthcare.thejobnetwork.com...,OffsiteApply,1.715496e+12,NaN,NaN,NaN,1.712904e+12,givinghomehealthcare.thejobnetwork.com,0,FULL_TIME,NaN,NaN,NaN,85701.0,4019.0,"Since 2012, Giving Home Health Care has helped..."
7733,3885851338,Baylor Scott & White Health,Care Manager RN,Job Summary\n\nThe Care Manager RN will be acc...,NaN,NaN,"Plano, TX",6250.0,6.0,NaN,NaN,Full-time,NaN,1.712275e+12,NaN,https://www.linkedin.com/jobs/view/3885851338/...,https://jsv3.recruitics.com/redirect?rx_cid=33...,OffsiteApply,1.714969e+12,NaN,Mid-Senior level,NaN,1.712377e+12,jsv3.recruitics.com,0,FULL_TIME,NaN,NaN,NaN,75023.0,48085.0,Job Summary\n\nThe Care Manager RN will be acc...
95505,3904931244,Colorado Housing and Finance Authority,Loan Servicing Customer Service/Cashier,position Loan Servicing Customer Service/Cashi...,60000.0,YEARLY,"Denver, CO",35904.0,1.0,NaN,50000.0,Full-time,NaN,1.713450e+12,NaN,https://www.linkedin.com/jobs/view/3904931244/...,http://www.chfainfo.com/careers,OffsiteApply,1.716042e+12,NaN,NaN,NaN,1.713450e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,80202.0,8031.0,position Loan Servicing Customer Service/Cashi...
49943,3901364196,Ageility,Physical Therapy Assistant,"Work With Purpose\n\nAt Ageility, we bring the...",NaN,NaN,"Fort Myers, FL",11227630.0,3.0,NaN,NaN,Part-time,NaN,1.713455e+12,NaN,https://www.linkedin.com/jobs/view/3901364196/...,https://careers.ageility.com/job-detail/physic...,OffsiteApply,1.716047e+12,NaN,Entry level,NaN,1.713455e+12,careers.ageility.com,0,PART_TIME,NaN,NaN,NaN,33900.0,NaN,"Work With Purpose\n\nAt Ageility, we bring the..."
81728,3903874384,MedStar Heal

In [32]:
df_select = pd.concat([df_ds, df_others])

print(f"There're totally {len(df_select)} samples for inserting VectorDB.")

There're totally 800 samples for inserting VectorDB.


# Create VectorDB

## Create Indexer: Construct the Container (aka. collection) for VectorDB

In [33]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [34]:
import chromadb
collection_name = "collection_postings"

persistent_client = chromadb.PersistentClient()
# print("check all functions/attribute for Chroma:\n", dir(persistent_client))



In [35]:
if not persistent_client.list_collections():
    print('empty')

In [36]:
import chromadb
collection_name = "collection_postings"

persistent_client = chromadb.PersistentClient()
# print("check all functions/attribute for Chroma:\n", dir(persistent_client))

if not persistent_client.list_collections():
    print(f"{collection_name} not in collection (DB) yet!")
else:
    if collection_name in persistent_client.list_collections()[0].name:
        print(f"{collection_name} is already in collection and would be deleted!")
        persistent_client.delete_collection(collection_name)

print(f"Create collection: {collection_name}!")  
vector_store = Chroma(
    client=persistent_client,
    collection_name=collection_name,
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  #save data locally, remove if not neccesary
)

#print("check all functions/attribute for Chroma:\n", dir(vector_store))

# if collection_name in vector_store.list_collections():
#     vector_store.delete_collection(collection_name)


collection_postings is already in collection and would be deleted!
Create collection: collection_postings!


## Indexing: Insert Data into VectorDB


In [37]:
from langchain_core.documents import Document
ids = []
documents = []
for index, row in df_select.iterrows():
    id_current = str(index)
    ids.append(id_current)
    
    min_salary = row['min_salary']
    if pd.isna(min_salary):
        min_salary = 0  # or any default value you prefer
    
    document_current = Document(
        page_content=row['job_post'],
        metadata={"title": row['title'],
                  "location": row['location'],
                  "min_salary": min_salary,
                  "pay_period": row['pay_period'],
                  "job_posting_url": row['job_posting_url'],
                 },
        id=row['job_id'],
    )
    documents.append(document_current)
print(f"There are {len(documents)}.\n")   
print(f'Example document content:\n{document_current}')

There are 800.

Example document content:
page_content='We consider applications for this position on an ongoing basis.\n\nOverview\n\nExperience a place of energy, passion, and excitement. A place where the joy of discovery and uncommon artistry blend to create exhilarating buying experiences—for true beauty enthusiasts. At Ulta Beauty, we’re transforming the world one shade, one lash, one cut at a time. Because beauty is powerful.\n\nIf you seek greater purpose—a place of vision, mission, and lived values—where voices are heard, contributions valued and recognized and growth opportunities abound, consider Ulta Beauty. Nowhere else are the possibilities quite this beautiful.\n\nTHE IMPACT YOU CAN HAVE:\n\nAs a Prestige Beauty Advisor at Ulta Beauty, you will have the opportunity to help our guests discover the truest expressions of their personal beauty. You will assist and consult. Explore and discover. And journey alongside our guests in ways that elevate the Ulta Beauty brand throu

In [38]:
vector_store.add_documents(documents=documents, ids=ids)

['3660',
 '75605',
 '86454',
 '20815',
 '117069',
 '72933',
 '6733',
 '55936',
 '32838',
 '113766',
 '46511',
 '92604',
 '64418',
 '74464',
 '24895',
 '106560',
 '28712',
 '27191',
 '67493',
 '2658',
 '115052',
 '6777',
 '5826',
 '90659',
 '108804',
 '38353',
 '40572',
 '51898',
 '53077',
 '115686',
 '48539',
 '18474',
 '16032',
 '2887',
 '119871',
 '22859',
 '24407',
 '117352',
 '23944',
 '31341',
 '56756',
 '83406',
 '111699',
 '50861',
 '98453',
 '74257',
 '46390',
 '1044',
 '31316',
 '73408',
 '7278',
 '47636',
 '22721',
 '117455',
 '41797',
 '115363',
 '107639',
 '19520',
 '74873',
 '61620',
 '6184',
 '3733',
 '81942',
 '47389',
 '71189',
 '51403',
 '96010',
 '53326',
 '29957',
 '91845',
 '121744',
 '10157',
 '31409',
 '49643',
 '86052',
 '86126',
 '41468',
 '27957',
 '47867',
 '69978',
 '100526',
 '48648',
 '96228',
 '74972',
 '41919',
 '14794',
 '13312',
 '51409',
 '116213',
 '58443',
 '104053',
 '24094',
 '113346',
 '101917',
 '96430',
 '97499',
 '49846',
 '23310',
 '29166',
 '

## Search: VectorDB as Retriever

### Vector Search (Similarity Search)
According to the query, find k=5 best possible job posts

In [39]:
query = "I am a passionate job seeker with a strong desire to embark on a career in data science. Having recently graduated with a degree in Computer Science, I have honed my skills in Python programming and developed a deep interest in machine learning. During my studies, I immersed myself in various projects that allowed me to apply these skills, from building predictive models to analyzing large datasets. My journey into data science has been driven by a fascination with uncovering hidden patterns in data and using these insights to solve real-world problems. I am now seeking an entry-level data scientist role where I can leverage my Python expertise and enthusiasm for machine learning to contribute to a dynamic team, learn from experienced professionals, and continue to grow my skills in this exciting field."

In [42]:
results = vector_store.similarity_search_with_score(
    query , k=3,
)
i =1
for res, score in results:
    print(f"* [{i}][SIM={score:3f}] {res.metadata['title']}\n---------------------\n \
          {res.page_content} \n--------------------\n \
           [{res.metadata}]\n\n")
    i +=1

* [1][SIM=0.440747] Lead Data Scientist
---------------------
           Lead Data Scientist – 3 Month Contract – Remote, United States – W2 ONLY, NO C2C

Join our client as a Lead Data Scientist, where you’ll collaborate with customer stakeholders to design and implement machine learning methods. Work within a large hybrid team, utilizing cutting-edge niche technology in the risk and compliance domain.

Responsibilities:

Analyze complex datasets and develop machine learning models for predictive analytics, classification, clustering, and optimization.Design algorithms inspired by genetics and evolution principles for problem-solving.Develop custom algorithms tailored to business needs and research objectives.Clean, preprocess, and normalize data for analysis, ensuring data quality and integrity.Deploy machine learning models into production environments and integrate them into existing software systems.


Qualifications:

Bachelor’s degree or higher in Computer Science, Data Science,

### Vector Search with Filtering 1
According to the query, find k=5 best possible job posts which have minimum salary greater than (gt) 100000

In [41]:
results = vector_store.similarity_search_with_score(
    query , k=3, filter={"min_salary": {"$gt": 100000}}
)# perator: $gt, $gte, $lt, $lte, $ne, $eq, $in, $nin
i = 1
for res, score in results:
    print(f"* [{i}][SIM={score:3f}] {res.metadata['title']}\n---------------------\n \
          {res.page_content} \n--------------------\n \
           [{res.metadata}]\n\n")
    i +=1

* [1][SIM=0.482590] Data Scientist - AI Investment
---------------------
           Are you interested in revolutionising the future of AI investment?
My client is looking for a data scientist to tackle intricate business challenges through advanced analytics and machine learning techniques.
You will take charge of both technical prowess, overseeing the creation, implementation, and upkeep of sophisticated machine learning models and algorithms, including extensive language models.
This role offers an exceptional chance to make a substantial impact and establish yourself as a visionary in the realms of data science and AI.
Responsibilities:You'll spearhead the development and implementation of groundbreaking AI and data science solutions.Steering the strategic path of the data science community, remaining at the forefront of applied AI and AI research.Effectively communicating with stakeholders and influencing decision-making.Overseeing project delivery from inception to deployment, en